In [2]:
:dep ode_solvers = "0.4.0"
:dep plotters = { version = "^0.3.5", default_features = false, features = ["evcxr", "all_series", "all_elements"] }

In [3]:
extern crate ode_solvers;
extern crate plotters;
use ode_solvers::*;
use plotters::prelude::*;

In [4]:
type State = Vector1<f64>;
type Time = f64;

struct Cs {
    beta0: f64,
    gamma: f64,
    k: f64,
    n: f64,
    ks: f64,
    ns: f64,
    s: f64,
}

impl ode_solvers::System<f64, State> for Cs {
    fn system(&self, t: Time, y: &State, dy: &mut State) {
        dy[0] = autorepressive(
            y[0], t, self.beta0, self.gamma,
	    self.k, self.n,
	    self.ks, self.ns, self.s,
        );
    }
}

fn autorepressive(
    x: f64,
    t: Time,
    beta0: f64,
    gamma: f64,
    k: f64,
    n: f64,
    ks: f64,
    ns: f64,
    s: f64,
) -> f64 {
    beta0 * (s / ks).powf(ns) / (1.0 + (s / ks).powf(ns)) / (1.0 + (x / k).powf(n)) - gamma * x	
}

let points1 = {
    let step_size = 0.05;
    let y0 = State::new(0.0);

    let system = Cs {
        beta0: 100.0,
        gamma: 1.0,
        k: 1.0,
        n: 1.0,
        ns: 10.0,
        ks: 0.1,
        s: 100.0,
    };
    // 0-10 200
    let mut stepper = Rk4::new(system, 0.0, y0, 10.0, step_size);
    let res  = stepper.integrate();
    let (x_out, y_out) = stepper.results().get();
    x_out.iter().zip(y_out.iter())
	.map(|(x,y)|(*x,y[0]))
	.collect::<Vec<(f64,f64)>>()
};

In [5]:
fn unregulated(
    t: f64,
    beta0: f64,
    gamma: f64,
    k: f64,
    n: f64,
    ks: f64,
    ns: f64,
    s: f64,
) -> f64 {
    beta0 / gamma * (1.0 - (-gamma * t).exp())
}


let points2 = (0..200).map(|t|t as f64 * 0.05).map(|t|(t,unregulated(t,100.0,1.0,1.0,1.0,10.0,0.1,100.0)))
.collect::<Vec<(f64,f64)>>();

In [6]:

let figure = evcxr_figure((640, 480), |root| {
    root.fill(&WHITE)?;
    
    let mut chart = ChartBuilder::on(&root)
        .caption("Constant-input dynamics", ("sans-serif", 20).into_font())
        .margin(5)
        .x_label_area_size(30)
        .y_label_area_size(30)
        .build_cartesian_2d(0.0f64..10.0f64, 0.0f64..100.0f64).unwrap();
    
    chart.configure_mesh().draw().unwrap();

    chart
    .draw_series(LineSeries::new(
        points1,
        &BLUE,
    )).unwrap()
.label("autorepressive")
.legend(|(x, y)| PathElement::new(vec![(x, y), (x + 20, y)], &BLUE));


chart
    .draw_series(LineSeries::new(
        points2,
        &RED,
    ))?
.label("unregulated")
.legend(|(x, y)| PathElement::new(vec![(x, y), (x + 20, y)], &RED));

chart
    .configure_series_labels()
    .background_style(&WHITE.mix(0.8))
    .border_style(&BLACK)
    .draw().unwrap();


Ok(())
});
figure

Constant-input dynamics
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.0
 
 
 
10.0
 
 
 
20.0
 
 
 
30.0
 
 
 
40.0
 
 
 
50.0
 
 
 
60.0
 
 
 
70.0
 
 
 
80.0
 
 
 
90.0
 
 
 
100.0
 
 
 
 
0.0
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
6.0
 
 
 
7.0
 
 
 
8.0
 
 
 
9.0
 
 
 
10.0
 
 
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="35,444 37,434 40,430 43,427 46,424 49,422 52,420 55,418 58,417 61,416 64,415 67,414 70,413 73,412 76,411 79,411 82,410 85,410 88,409 91,409 94,409 97,408 100,408 103,408 106,407 109,407 112,407 115,407 118,407 121,407 124,406 127,406 130,406 133,406 136,406 139,406 142,406 145,406 148,406 151,406 154,406 157,406 160,406 163,406 166,406 169,406 172,405 175,405 178,405 181,405 184,405 187,405 190,405 193,405 196,405 199,405 202,405 205,405 208,405 211,405 214,405 217,405 220,405 223,405 226,405 229,405 232,405 235,405 238,405 241,405 244,405 247,405 250,405 253,405 256,405 259,405 262,405 265,405 268,405 271,405 274,405 277,405 280,405 283,405 286,405 289,405 292,405 295,405 298,405 301,405 304,405 307,405 310,405 313,405 316,405 319,405 322,405 325,405 328,405 331,405 334,405 337,405 340,405 343,405 346,405 349,405 352,405 355,405 358,405 361,405 364,405 367,405 370,405 373,405 376,405 379,405 382,405 385,405 388,405 391,405 394,405 397,405 400,405 403,405 406,405 409,405 412,405 415,405 418,405 421,405 424,405 427,405 430,405 433,405 436,405 439,405 442,405 445,405 448,405 451,405 454,405 457,405 460,405 463,405 466,405 469,405 472,405 475,405 478,405 481,405 484,405 487,405 490,405 493,405 496,405 499,405 502,405 505,405 508,405 511,405 514,405 517,405 520,405 523,405 526,405 529,405 532,405 535,405 538,405 541,405 544,405 547,405 550,405 553,405 556,405 559,405 562,405 565,405 568,405 571,405 574,405 577,405 580,405 583,405 586,405 589,405 592,405 595,405 598,405 601,405 604,405 607,405 610,405 613,405 616,405 619,405 622,405 625,405 628,405 631,405 634,405 "/>
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="35,444 37,424 40,405 43,387 46,370 49,353 52,337 55,323 58,308 61,295 64,282 67,270 70,258 73,247 76,237 79,227 82,217 85,208 88,199 91,191 94,183 97,176 100,169 103,162 106,156 109,150 112,144 115,139 118,133 121,128 124,124 127,119 130,115 133,111 136,107 139,103 142,100 145,96 148,93 151,90 154,87 157,85 160,82 163,80 166,77 169,75 172,73 175,71 178,69 181,67 184,65 187,64 190,62 193,61 196,59 199,58 202,57 205,55 208,54 211,53 214,52 217,51 220,50 223,49 226,48 229,48 232,47 235,46 238,45 241,45 244,44 247,43 250,43 253,42 256,42 259,41 262,41 265,40 268,40 271,39 274,39 277,39 280,38 283,38 286,38 289,37 292,37 295,37 298,37 301,36 304,36 307,36 310,36 313,35 316,35 319,35 322,35 325,35 328,35 331,34 334,34 337,34 340,34 343,34 346,34 349,34 352,34 355,33 358,33 361,33 364,33 367,33 370,33 373,33 376,33 379,33 382,33 385,33 388,33 391,33 394,33 397,32 400,32 403,32 406,32 409,32 412,32 415,32 418,32 421,32 424,32 427,32 430,32 433,32 436,32 439,32 442,32 445,32 448,32 451,32 454,32 457,32 460,32 463,32 466,32 469,32 472,32 475,32 478,32 481,32 484,32 487,32 490,32 493,32 496,32 499,32 502,32 505,32 508,32 511,32 514,32 517,32 520,32 523,32 526,32 529,32 532,32 535,32 538,32 541,32 544,32 547,32 550,32 553,32 556,32 559,32 562,32 565,32 568,32 571,32 574,32 577,32 580,32 583,32 586,32 589,32 592,32 595,32 598,32 601,32 604,32 607,32 610,32 613,32 616,32 619,32 622,32 625,32 628,32 631,32 "/>
 
 
 
autorepressive
 
 
unregulated